# <font color=blue>Image_processing_module </font>
Image_processing_module reads the images and applies image processing algorithms to obtain the plant phenotyping measurements.

### Importing modules
The Python and custom modules are loaded using the import command

In [6]:
import numpy as np
import cv2
import pandas as pd
import cropping
from datetime import datetime
import os
import colorCorrection
import imalgo
import segmentation
import sys
import plotting
import leafcount
from scipy import signal
from scipy.signal import find_peaks
import save

### Variable declaration
__expID__ is the experiment ID. __daySowing__ is the day when the seeds were sown into the soil. __dayStart__ is the first day of the image acquisition from the sowing day. __dayStop__ is the last day of the image acquisition from the sowing day. __krnl__ is the filter kernel size.

In [7]:
expID, daySowing, cam = 'Exp8', '2018-08-03-11-00', 'cam04'
winsiz = 300
win = int(winsiz/2)

### Experiment Directories
Each experiment has its own set of files which are stored in a specific directory. They are loaded based on the full file directory.

In [8]:
dirCurrent  = os.getcwd()
dirParent   = os.path.abspath(os.path.join(dirCurrent, os.pardir)) 
root        = os.path.join(dirParent, expID)
path_label  = os.path.join(root, expID + '_label.csv')
path_images = os.path.join(root, expID + '_' + 'images', cam)
path_CSV    = os.path.join(root, expID + '_csvFiles')
path_coor   = os.path.join(root, expID + '_tray_coord_' + cam + '.csv')
path_mtx    = os.path.join(root, 'camParam' , cam + '_MTX' + '.npy')
path_dis    = os.path.join(root, 'camParam' , cam + '_DIST' + '.npy')
path_trash  = os.path.join(dirParent, 'trash')

if not os.path.exists(path_CSV): os.makedirs(path_CSV) 

if not os.path.exists(path_trash): os.makedirs(path_trash) 

### Read files 
This section reads the plant images and csv files which contain the general information 

In [9]:
images = os.listdir(path_images)   
mtx = np.load(path_mtx)
dist = np.load(path_dis)
coor = pd.read_csv(path_coor)
coor = coor[coor['cam']== int(cam[3:])]
card = cv2.imread('card.png')
label = pd.read_csv(path_label)
label['col'] = label['position'].apply(lambda x : x[0])
label['row'] = label['position'].apply(lambda x : x[1])
trayCol = pd.unique(label.loc[:, 'col'])
trayRow = pd.unique(label.loc[:, 'row'])
potCoor = cropping.potAll( coor, trayCol, trayRow, win)  

This section filters out the outliers plants based on the ExpX_label.csv file/

In [10]:
condition = pd.DataFrame(columns = ['tray'])
cntAll = 0

cntAux, end, step = 0, 11, 1
for mainCnt in range(cntAux,len(images)):
# for mainCnt in range(cntAux, end, step):
# for mainCnt in range(cntAux,cntAux+1):
    imName = images[mainCnt]
    date = datetime.strptime(imName.split('_')[-1].split('.')[0], '%Y-%m-%d-%H-%M-%S')
    dateSowing = datetime.strptime(daySowing, '%Y-%m-%d-%H-%M')
    dateDelta = date - dateSowing
    mins = dateDelta.total_seconds()/60
    
    # print('Images Name =',imName,'Day=',dateDelta.days , 'mins =',mins, 'Number=',mainCnt) 

    imPath = os.path.join(path_images,imName)                                                        # Image full path
    imsrc = cv2.imread(imPath,cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)            # Read Image raw format
    
    if not type(imsrc) is np.ndarray: continue

    h,  w = imsrc.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    mapx, mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
    imLens = cv2.remap(imsrc,mapx,mapy,cv2.INTER_LINEAR)
    tARow, tACol, tAH, tAW = cropping.trayAll(coor)
    trays = imLens[tARow : tARow + tAH, tACol : tACol + tAW]

    imLensRGB = cv2.cvtColor(imLens, cv2.COLOR_BGR2RGB)
    colorTrue, colorImg = colorCorrection.colorValues(imLensRGB, coor, trayCol, trayRow, label, card)
    alpha, beta = colorCorrection.fit(colorTrue, colorImg)
    imCorrRGB = colorCorrection.correct(imLensRGB, alpha, beta)
    imCorrBGR = cv2.cvtColor(imCorrRGB, cv2.COLOR_RGB2BGR)

    
    # plotting.im(imLens, 'imLens ' + 'day=' + str(dateDelta.days))
    # plotting.im(imCorrBGR, 'imCorrBGR ' + 'day=' + str(dateDelta.days))
    # cv2.imwrite(os.path.join(path_trash, 'Correc' + imName), imCorrBGR)
    # cv2.imwrite(os.path.join(path_trash, imName), imLens)
 
    
    # for cntP in range(30):
    for cntP in range(len(potCoor)):
        trayNum, posn, pTop, pLeft, pcRow, pcCol, pixelSiz = potCoor.loc[cntP, :].values
        trayName = 'T'+ str(trayNum).zfill(2)
        potName = label["ecotype"][(label["tray"] == trayNum) & 
                                (label["position"]==posn)].values.tolist()[0]
        
        if potName == 'Checker': continue 
        if potName == "Null" : continue 

        
        potCrp = imCorrBGR[pTop : pcRow+win, pLeft: pcCol + win]
        off = np.int(np.mean(potCrp.shape[0:2])*0.30)
        potAux = potCrp[off:(potCrp.shape[0]) - off,off:(potCrp.shape[1]) - off] 
        mask = imalgo.grosSegm(potAux, 21)
        pixels = len(mask[mask > 0])
        sid = int((1.5) * np.sqrt(pixels))
        # plotting.im(mask ,'mask ')
    
        if pixels <= 10: continue
    
        # print(10*'-')
    
        mTop, mLeft, mcRow, mcCol = cropping.mask(mask, pcRow, pcCol, win, off)
        # pot = imCorrBGR[mTop : mcRow + win, mLeft : mcCol + win]
        pot = imCorrBGR[mcRow - sid : mcRow + sid, mcCol - sid : mcCol + sid]
        
        maskSeg, potSeg =  segmentation.segment(pot)
        
        if len(maskSeg[maskSeg>0]) == 0: continue
        
        cntr, rosPheno = imalgo.pheno(maskSeg)
        area= len(np.where(maskSeg==255)[0])
        areamm2 = area * (pixelSiz ** 2)

        # plotting.imT(pot, potSeg, trayName + posn, 'potSeg ' + str(off))
        
        
        rosSig,rosXY,rosMeanX,rosMeanY = leafcount.radialTrans(maskSeg)
        rosSig = np.roll(rosSig, - np.argmin(rosSig))
        num, den = signal.ellip(8, 0.01, 120, 0.125)              # Elliptic Filter Coefficient num, (Order, ripple, atte, )
        fgust = signal.filtfilt(num, den, rosSig, method="gust")
        fgust2 = np.max(fgust) -  fgust
        peaks, _ = find_peaks(fgust, height=0)
        peaks2, _ = find_peaks(fgust2, height=0)
        fileID = potName, posn, trayName, cam, expID, mins, date
       
        save.CVS(fileID,path_CSV,areamm2,rosPheno,len(peaks))



KeyboardInterrupt: 